In [2]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = ['2023-24', '2022-23', '2021-22']

# Initialize an empty list to store game IDs
all_game_ids = []

# Loop through each season and collect game IDs
for season in seasons:
    game_finder = leaguegamefinder.LeagueGameFinder(season_nullable=season)
    games = game_finder.get_data_frames()[0]
    game_ids = games['GAME_ID'].unique()
    all_game_ids.extend(game_ids)

In [3]:
len(all_game_ids)

6441

In [11]:
# TODO: need to filter out some players
from nba_api.stats.static import players
player_id_set = {item["id"] for item in players.get_active_players() if "id" in item}
len(player_id_set)

615

1.	2-point unassisted
2.	2-point assisted
3.	3-point unassisted
4.	3-point assisted
5.	4-point play unassisted
6.	4-point play assisted

7. Missed (blocked)
8. Missed (unblocked)

9. Turnover (stolen)
10. Turnover (unforced or not stolen)

11. Possession ends with 0 FTs made
12. Possession ends with 1 FTs made
13. Possession ends with 2 FTs made
14. Possession ends with 3 FTs made

In [ ]:
import nba_on_court as noc
from nba_api.stats.endpoints import playbyplayv2
import pandas as pd
import numpy as np

# Define the event message types that matter
FIELD_GOAL_MADE = 1
FIELD_GOAL_MISSED = 2
FREE_THROW_ATTEMPT = 3
REBOUND = 4
TURNOVER = 5

valid_event_types = {FIELD_GOAL_MADE, FIELD_GOAL_MISSED, FREE_THROW_ATTEMPT, REBOUND, TURNOVER}

# valid_event_types = {FIELD_GOAL_MADE, FREE_THROW_ATTEMPT}

columns = [
    "OFF_PLAYER1_ID", "OFF_PLAYER2_ID", "OFF_PLAYER3_ID", "OFF_PLAYER4_ID", "OFF_PLAYER5_ID",
    "DEF_PLAYER1_ID", "DEF_PLAYER2_ID", "DEF_PLAYER3_ID", "DEF_PLAYER4_ID", "DEF_PLAYER5_ID",
    "OUTCOME",         # One of the 12 categories described (e.g., "2PT_ASSIST", "MISSED_BLOCKED", etc.)
    "SECOND_CHANCE",    # 0 or 1
    "SHOOTER_ID",       # ID of the player who took the shot (if applicable)
    "ASSISTER_ID",      # ID of the player who assisted (if applicable)
    "BLOCKER_ID",       # ID of the player who blocked the shot (if applicable)
    "STEALER_ID",       # ID of the player who stole the ball (if applicable)
    "REBOUNDER_ID"      # ID of the player who rebound the ball (if applicable)
]

pbp = playbyplayv2.PlayByPlayV2(game_id='0022100001').play_by_play.get_data_frame()
pbp = noc.players_on_court(pbp)
filtered_pbp = pbp[pbp['EVENTMSGTYPE'].isin(valid_event_types)].copy()

# filtered_pbp.to_csv("goal_made.csv")

# Now we have only the rows we care about.
# We can iterate through these rows and handle each event type as needed.

processed_pbp = pd.DataFrame(columns=columns)
rows = []

def get_off_def_players(row):
    away_players = [row['AWAY_PLAYER1'], row['AWAY_PLAYER2'], row['AWAY_PLAYER3'], row['AWAY_PLAYER4'], row['AWAY_PLAYER5']]
    home_players = [row['HOME_PLAYER1'], row['HOME_PLAYER2'], row['HOME_PLAYER3'], row['HOME_PLAYER4'], row['HOME_PLAYER5']]
    
    if row['PERSON1TYPE'] == 5:
        # Away is offense
        off_players = away_players
        def_players = home_players
    elif row['PERSON1TYPE'] == 4:
        # Home is offense
        off_players = home_players
        def_players = away_players
    else:
        # Unexpected case
        off_players = [np.nan]*5
        def_players = [np.nan]*5
    
    return off_players, def_players

def check_and_one(pbp, current_idx, shooter_id):
    # Look ahead for next free throw attempt by the same shooter
    next_events = pbp.iloc[current_idx+1:]
    for i, r in next_events.iterrows():
        if r['EVENTMSGTYPE'] == FREE_THROW_ATTEMPT and r['PLAYER1_ID'] == shooter_id:
            # Check if made free throw
            desc = (r['HOMEDESCRIPTION'] if pd.notnull(r['HOMEDESCRIPTION']) else '') + \
                   (r['VISITORDESCRIPTION'] if pd.notnull(r['VISITORDESCRIPTION']) else '')
            if "MISS" not in desc.upper():
                # FT made => and-one
                return True
            else:
                return False
    return False

for idx, row in filtered_pbp.iterrows():
    event_type = row['EVENTMSGTYPE']
    
    # Switch-like structure to handle cases
    if event_type == FIELD_GOAL_MADE:
        # Determine offense/defense
        off_players, def_players = get_off_def_players(row)

        # Determine if 2pt or 3pt
        desc = (row['HOMEDESCRIPTION'] if pd.notnull(row['HOMEDESCRIPTION']) else '') + \
               (row['VISITORDESCRIPTION'] if pd.notnull(row['VISITORDESCRIPTION']) else '')
        desc_upper = desc.upper()
        is_3pt = "3PT" in desc_upper

        # Determine assisted or unassisted
        assisted = (row['PERSON2TYPE'] == 4 or row['PERSON2TYPE'] == 5)

        # Base outcome mapping
        # 0: 2-point unassisted
        # 1: 2-point assisted
        # 2: 3-point unassisted
        # 3: 3-point assisted
        if not is_3pt:
            base_outcome = 1 if assisted else 0
        else:
            base_outcome = 3 if assisted else 2

        shooter_id = row['PLAYER1_ID']
        assister_id = row['PLAYER2_ID'] if assisted else np.nan

        # Check for and-one
        if check_and_one(filtered_pbp, idx, shooter_id):
            # 4-point scenarios for both 2pt+1 or 3pt+1
            # If base_outcome in [0,2] => unassisted => outcome=4
            # If base_outcome in [1,3] => assisted => outcome=5
            if base_outcome in [0, 2]:
                final_outcome = 4
            else:
                final_outcome = 5
        else:
            final_outcome = base_outcome

        data = {
            "OFF_PLAYER1_ID": off_players[0],
            "OFF_PLAYER2_ID": off_players[1],
            "OFF_PLAYER3_ID": off_players[2],
            "OFF_PLAYER4_ID": off_players[3],
            "OFF_PLAYER5_ID": off_players[4],
            "DEF_PLAYER1_ID": def_players[0],
            "DEF_PLAYER2_ID": def_players[1],
            "DEF_PLAYER3_ID": def_players[2],
            "DEF_PLAYER4_ID": def_players[3],
            "DEF_PLAYER5_ID": def_players[4],
            "OUTCOME": final_outcome,
            "SECOND_CHANCE": 0, # Not computed here
            "SHOOTER_ID": shooter_id,
            "ASSISTER_ID": assister_id,
            "BLOCKER_ID": np.nan,
            "STEALER_ID": np.nan,
            "REBOUNDER_ID": np.nan
        }
        rows.append(data)

    elif event_type == FIELD_GOAL_MISSED:
        # Process missed shot
        # Identify if it was blocked or not from HOMEDESCRIPTION/VISITORDESCRIPTION
        pass
    elif event_type == FREE_THROW_ATTEMPT:
        # Process free throw attempt
        # Determine if it was made or missed, number of free throws, etc.
        pass
    elif event_type == REBOUND:
        # Process rebound event
        # Determine if it's offensive or defensive rebound
        pass
    elif event_type == TURNOVER:
        # Process turnover
        # Identify if it was a steal or unforced turnover
        pass
    else:
        # Should not reach here because we filtered out other types
        pass

processed_pbp = pd.DataFrame(rows, columns=columns)

In [ ]:
import nba_on_court as noc
from nba_api.stats.endpoints import playbyplayv2
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_game(game_id):
    pbp = playbyplayv2.PlayByPlayV2(game_id=game_id).play_by_play.get_data_frame()
    pbp = noc.players_on_court(pbp)

num_workers = 6

results = []
with ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = {executor.submit(process_game, game_id): game_id for game_id in all_game_ids}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Games"):
        try:
            results.append(future.result())
        except Exception as e:
            print(f"Error processing game {futures[future]}: {e}")


Processing Games:   1%|          | 50/6441 [00:03<07:39, 13.89it/s]

Error processing game 0042300126: 'resultSet'


Processing Games:   1%|▏         | 85/6441 [00:34<6:43:55,  3.81s/it]

Error processing game 0042300112: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
Error processing game 0042300141: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)


Processing Games:   1%|▏         | 92/6441 [00:35<2:17:13,  1.30s/it]

...

Processing Games:   2%|▏         | 130/6441 [00:40<13:29,  7.80it/s]

Error processing game 0022301158: 'resultSet'


Processing Games:   2%|▏         | 141/6441 [00:41<13:13,  7.94it/s]

Error processing game 0022301147: 'resultSet'


Processing Games:   2%|▏         | 157/6441 [00:45<30:19,  3.45it/s]


In [9]:
len(all_game_ids)

6441

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Number of players in the league
v = 615  
# Number of player vectors per possession
seq_len = 10

# Define model input
# Shape: (batch_size, seq_len, v)
input_players = Input(shape=(seq_len, v), name='players_input')

# TODO: Middle of the model
x = input_players
x = layers.Flatten()(x)

# Final output layers:
# 1) Main outcome (12 classes)
main_out = layers.Dense(12, activation='softmax', name='main_out')(x)

# 2) Second chance indicator (binary)
second_chance_out = layers.Dense(1, activation='sigmoid', name='second_chance_out')(x)

# Construct the model
model = Model(inputs=input_players, outputs=[main_out, second_chance_out])

# Compile the model with appropriate losses:
# - main_out is categorical: use 'categorical_crossentropy'
# - second_chance_out is binary: use 'binary_crossentropy'
model.compile(
    optimizer='adam',
    loss={
        'main_out': 'categorical_crossentropy',
        'second_chance_out': 'binary_crossentropy'
    },
    metrics={
        'main_out': 'accuracy',
        'second_chance_out': 'accuracy'
    }
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ players_input       │ (None, 10, 615)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 6150)      │          0 │ players_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ main_out (Dense)    │ (None, 12)        │     73,812 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ second_chance_out   │ (None, 1)         │      6,151 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 79,963 (312.36 KB)

 Trainable params: 79,963 (312.36 KB)

 Non-trainable params: 0 (0.00 B)